In [1]:
import pandas as pd
import torch
from modelV2.tune import HyperParamOptimizer
from modelV2.train import tuning_wrapper
# from modelV2.arch.rn_2 import ResNet18
from torchvision.transforms import v2
import os
import re
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

/home/tbrow51/.conda/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def extract_masked_path(target_mask_factor: str, mask_factors_str: str, masked_paths_str: str):
    try:
        # parse the mask factors/masked path strings
        mask_factors_list = parse_mask_factors(mask_factors_str)
        mask_paths_list = parse_masked_paths(masked_paths_str)

        # find the index of the target mask factor
        target_idx = mask_factors_list.index(target_mask_factor)

        # extract the target masked path
        return mask_paths_list[target_idx]
    
    except Exception as e:
        print(e)
        return 'CAUGHT_ERROR'

def parse_mask_factors(mask_factors_str: str):
    # remove brackets and spaces from the string
    mask_factors_str = mask_factors_str.translate({ord(c): None for c in "][ "})
    # the split on commas
    return mask_factors_str.split(',')

def parse_masked_paths(masked_paths_str: str):
    # split on dollar characters (pls dont have any of these in your paths!!)
    return masked_paths_str.split('$')

In [3]:
SEED = 49713
BASE_DIR = "/mnt/NAS3/DataBalance/balance-study/data/images/run_e1_a_1200_800/"
MASK_STR = "0_0"
BALANCE_DISTRIBUTIONS = False

N_BOOTSTRAPS = 5
EPOCHS_PER_RUN = 20
MONITOR_METRIC = 'loss'
HPARAM_RANGE_DICT = {
    'learning_rate': (0.00001, 0.01),
}


BATCH_SIZE = 48
N_RANDOM = 5
N_GUIDED = 10
MODEL_NAME = f"masked_{MASK_STR}_rn18_r3_a"

In [4]:
pos_df = pd.read_csv(os.path.join(BASE_DIR, "pos_out_df.csv"))
neg_df = pd.read_csv(os.path.join(BASE_DIR, "neg_proc_df.csv"))

# only include exams we've assigned to the tuning set
# this is done so evaluation class balance is consistent 
# between train/val/test sets
# 10/90 balance testing can be done after tuning is completed
neg_df = neg_df[neg_df["include_in_tuning"] == 1]

pos_df.dropna(subset=['masked_factors', 'masked_png_paths'], inplace=True)
neg_df.dropna(subset=['masked_factors', 'masked_png_paths'], inplace=True)
print(pos_df[['assigned_split']].value_counts())
print(neg_df[['assigned_split']].value_counts())

def correct_paths_to_kraken(hiti_path):
    replace_str = r"/mnt/NAS3/DataBalance/balance-study\1"
    # use re.sub to replace everything before '/data/images'
    return re.sub(r'^.*?(/data/images)', replace_str, hiti_path)

pos_df['__target_path'] = pos_df.apply(
    lambda x: correct_paths_to_kraken(extract_masked_path(
        MASK_STR.replace('_', '.'), 
        x.masked_factors, 
        x.masked_png_paths
    )), 
    axis=1
)

neg_df['__target_path'] = neg_df.apply(
    lambda x: correct_paths_to_kraken(extract_masked_path(
        MASK_STR.replace('_', '.'), 
        x.masked_factors, 
        x.masked_png_paths
    )), 
    axis=1
)

DF_POOL_DICT = {
    "pos": {
        "train": pos_df[pos_df.assigned_split == "train"],
        "val": pos_df[pos_df.assigned_split == "val"],
        "test": pos_df[pos_df.assigned_split == "test"]
    },
    "neg": {
        "train": neg_df[neg_df.assigned_split == "train"],
        "val": neg_df[neg_df.assigned_split == "val"],
        "test": neg_df[neg_df.assigned_split == "test"]
    }
}

assigned_split
train             4485
test               986
val                955
Name: count, dtype: int64
assigned_split
train             45232
val                9743
test                991
Name: count, dtype: int64


In [5]:
AUGMENT_DICT = {
    'crop': {
        'enabled': True, 
        'func': v2.RandomResizedCrop,
        'prob': 1.0,
        'params': {
            'size': (1200, 800), 
            'scale': (0.7, 1.0),
        }
    },
    'rotation': { 
        'enabled': True, 
        'func': v2.RandomRotation,
        'prob': 0.4,
        'params': {
            'degrees': 5
        }
    },
    'color_jitter': {
        'enabled': True,
        'func': v2.ColorJitter,
        'prob': 0.2,
        'params': {
            'brightness': 0.4, 
            'contrast': 0.4,
            'saturation': 0.4, 
            'hue': 0.2
        }
    },
    'gaussian_blur': {
        'enabled': True,
        'func': v2.GaussianBlur,
        'prob': 0.2,
        'params': {
            'kernel_size': 3
        }
    }
}

In [6]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"using {device} device...")

using cuda device...


In [7]:
# get optimizer
hparam_optimizer = HyperParamOptimizer(
    hparam_range_dict=HPARAM_RANGE_DICT, 
    balance=BALANCE_DISTRIBUTIONS,
    monitor_metric=MONITOR_METRIC,
    n_bootstraps=N_BOOTSTRAPS,
    epochs_per_run=EPOCHS_PER_RUN,
    seed=SEED
)

# set device and model
hparam_optimizer.set_model(
    device=device, 
    model_type='resnet18'
)

# load data into optimizer
hparam_optimizer.load_data(
    df_pool_dict=DF_POOL_DICT,
    batch_size=BATCH_SIZE,
    augment_dict=AUGMENT_DICT
)

# start optimizer
hparam_optimizer.optimize(
    objective=tuning_wrapper,
    n_random=N_RANDOM, 
    n_guided=N_GUIDED, 
    model_name=MODEL_NAME
)


Data loaded...

Optimizing model ------------------------------------------------------------
Logging results to './logs/masked_0_0_rn18_r3_a_opt.json'
|   iter    |  target   | learni... |
-------------------------------------
running 5 bootstraps...
with seeds: [4266198 9960322 3226871 5341026  569431]


/mnt/NAS3/DataBalance/balance-study/mammo-balance-study/modelV2/data.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_df.loc[:, '__class'] = 1
/mnt/NAS3/DataBalance/balance-study/mammo-balance-study/modelV2/data.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_df.loc[:, '__class'] = 0
/home/tbrow51/.conda/envs/torch/lib/python3.9/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")



Epoch 0 ----------------------------------------


100%|██████████| 28/28 [00:18<00:00,  1.51batch/s, val_loss=0.691, val_acc=0.532, val_auc=0.56, val_f1=0.659, val_prec=0.518, val_rec=0.906]


saving model with best val_loss '0.6912'...

Epoch 1 ----------------------------------------


...


Epoch 19 ----------------------------------------


100%|██████████| 28/28 [00:20<00:00,  1.40batch/s, val_loss=0.687, val_acc=0.551, val_auc=0.58, val_f1=0.598, val_prec=0.541, val_rec=0.669]


saving model with best val_loss '0.6866'...
best model loaded...


100%|██████████| 28/28 [00:19<00:00,  1.45batch/s, val_loss=0.687, val_acc=0.554, val_auc=0.577, val_f1=0.596, val_prec=0.545, val_rec=0.657]


KeyError: 'test_loss'